# Lyric LLM RAG Project - Harshini Raju
**PART 1 - Scrape Data and Form Dataset**
<br>Scrape Lyrics to all  BTS Songs from doolsetbangtan.com

## Installation

In [67]:
%pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv
%pip install -q langchain
%pip install -q pypdf
%pip install -q torch
%pip install -q transformers
%pip install -q faiss-cpu
%pip install -q chromadb
%pip install -q requests
%pip install -q beautifulsoup4
%pip install -q nltk

Note: you may need to restart the kernel to use updated packages.


## Extract information from doolset bangtan

In [21]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import re
from urllib.parse import urlparse
import json
import codecs

In [22]:

# url = 'https://doolsetbangtan.wordpress.com/bts-albums/'
url = 'https://doolsetbangtan.wordpress.com/solo-projects/'

def is_valid_https_link(link):
    parsed_url = urlparse(link)
    return parsed_url.scheme == 'https' and parsed_url.netloc

def get_sublinks(link):
    response = requests.get(link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        sublinks = soup.find_all('a')
        return [urljoin(link, sublink.get('href')) for sublink in sublinks if is_valid_https_link(sublink.get('href'))]
    else:
        print(f"Failed to retrieve sublinks from {link}. Status code: {response.status_code}")
        return []

# Send an HTTP request to the main URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract all the links on the main page
    links = soup.find_all('a')

    # Iterate through each link and get sublinks for valid HTTPS links
    for link in links:
        main_link = urljoin(url, link.get('href'))
        
        # Check if the main link is a valid HTTPS link
        if is_valid_https_link(main_link):
            print(main_link)
            
            # # Get sublinks for the main link
            # sublinks = get_sublinks(main_link)
            # print(f"Sublinks: {sublinks}")
else:
    print(f"Failed to retrieve the main page. Status code: {response.status_code}")

https://doolsetbangtan.wordpress.com/solo-projects/#content
https://doolsetbangtan.wordpress.com
https://doolsetbangtan.wordpress.com/bts-albums/
https://doolsetbangtan.wordpress.com/solo-projects/
https://doolsetbangtan.wordpress.com/soundcloud/
https://doolsetbangtan.wordpress.com/miscellaneous/
https://twitter.com/doolsetbangtan
https://www.reddit.com/user/diminie
https://doolsetbangtan.wordpress.com/solo-projects/
https://doolsetbangtan.wordpress.com/
https://doolsetbangtan.wordpress.com/d-day/
https://doolsetbangtan.wordpress.com/face/
https://doolsetbangtan.wordpress.com/indigo/
https://doolsetbangtan.wordpress.com/2022/10/28/jin-the-astronaut/
https://doolsetbangtan.wordpress.com/jack-in-the-box/
https://doolsetbangtan.wordpress.com/d-2/
https://doolsetbangtan.wordpress.com/mono/
https://doolsetbangtan.wordpress.com/hope-world/
https://doolsetbangtan.wordpress.com/agust-d/
https://doolsetbangtan.wordpress.com/rm/
https://doolsetbangtan.wordpress.com/solo-projects/?share=twitter


In [24]:
song_links =[]

def is_valid_https_link(link):
    parsed_url = urlparse(link)
    return parsed_url.scheme == 'https' and parsed_url.netloc

pattern = re.compile(r'https://doolsetbangtan\.wordpress\.com/20.*')

for url in bts_albums:
    # Send an HTTP request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the div with class "entry-content"
        entry_content_div = soup.find('div', class_='entry-content')

        if entry_content_div:
            # Extract links from the div
            links = entry_content_div.find_all('a')

            # Print the extracted links
            for link in links:
                if is_valid_https_link(link.get('href')) and pattern.match(link.get('href')):
                    # print(link.get('href'))
                    song_links.append(link.get('href'))
                else:
                    continue
        else:
            print("No div with class 'entry-content' found.")
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")


**Cleaning the links to only include the links to song lyrics**

In [25]:
filtered_song_links = [link for link in song_links if not any(keyword in link for keyword in ['twitter', 'facebook', 'share'])]

In [26]:
filtered_song_links

['https://doolsetbangtan.wordpress.com/2018/06/01/born-singer/',
 'https://doolsetbangtan.wordpress.com/2018/06/21/no-more-dream/',
 'https://doolsetbangtan.wordpress.com/2018/12/15/n-o/',
 'https://doolsetbangtan.wordpress.com/2019/04/10/boy-in-luv/',
 'https://doolsetbangtan.wordpress.com/2019/01/07/danger/',
 'https://doolsetbangtan.wordpress.com/2018/07/02/i-need-u/',
 'https://doolsetbangtan.wordpress.com/2018/07/02/run/',
 'https://doolsetbangtan.wordpress.com/2018/07/27/fire/',
 'https://doolsetbangtan.wordpress.com/2018/06/20/blood-sweat-tears/',
 'https://doolsetbangtan.wordpress.com/2018/06/28/spring-day/',
 'https://doolsetbangtan.wordpress.com/2018/06/30/dna/',
 'https://doolsetbangtan.wordpress.com/2018/06/01/fake-love/',
 'https://doolsetbangtan.wordpress.com/2018/08/24/idol/',
 'https://doolsetbangtan.wordpress.com/2019/04/13/boy-with-luv/',
 'https://doolsetbangtan.wordpress.com/2020/02/21/on/',
 'https://doolsetbangtan.wordpress.com/2020/11/20/life-goes-on/',
 'https:/

## Extracting the lyric for each song from the links 

In [66]:
song_data = {}

In [67]:
for url in filtered_song_links:
    # print(url)
    # Parse the URL
    parsed_url = urlparse(url)

    # Get the path from the URL
    path = parsed_url.path

    # Split the path and get the last part
    parts = path.split('/')
    song_name = parts[-2]
    # Send an HTTP request to the URL
    response = requests.get(url)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        paragraphs = soup.find_all('p')
        print(song_name)
        # Decode Unicode escape sequences for each paragraph
        decoded_paragraphs = ' '.join([codecs.decode(paragraph.text, 'unicode_escape') for paragraph in paragraphs])

        # Store the decoded paragraphs as a single string in the dictionary
        song_data[song_name] = decoded_paragraphs.strip()
    else:
        print(f"Failed to retrieve the page. url: {url} Status code: {response.status_code}")

born-singer
no-more-dream
n-o
boy-in-luv
danger
i-need-u
run
fire
blood-sweat-tears
spring-day
dna
fake-love
idol
boy-with-luv
on
life-goes-on
yet-to-come
run-bts
persona
stay
moon
jamais-vu
trivia-seesaw
bts-cypher-pt-3-killer
outro-ego
outro-her
filter
friends
intro-singularity
zero-oclock
euphoria
dimple
jump
boy-in-luv
i-need-u
boyz-with-fun
tony-montana-feat-yankie
epilogue-young-forever
spring-day
dna
epiphany
trivia-seesaw
still-with-you-jungkook
for-youth
life-goes-on
fly-to-my-room
blue-and-grey
%ec%9e%a0%ec%8b%9c-telepathy
dis-ease
stay
persona
boy-with-luv
make-it-right
jamais-vu
dionysus
interlude-shadow
black-swan
filter
my-time
louder-than-bombs
on
ugh
zero-oclock
inner-child
friends
moon
respect
we-are-bulletproof-the-eternal
outro-ego
on
persona
boy-with-luv
mikrokosmos
make-it-right
home
jamais-vu
dionysus
01
trivia-just-dance
serendipity
dna
dimple
trivia-love
outro-her
intro-singularity
fake-love
the-truth-untold-ft-steve-aoki
trivia-seesaw
outro-tear
epiphany
im-fin

In [68]:
song_data

{'born-singer': 'Â\xa0 Original Track: Born Sinner byÂ\xa0J.Cole\nLyrics by RM, SUGA, j-hope . Â\xa0 Iâ\x80\x99m a born singer ì¢\x80 ë\x8a¦ì\x96´ë²\x84ë¦° ê³\xa0ë°± (I swear)\nIâ\x80\x99m a born singer, a bit belated confessionÂ\xa0 (I swear) ì\x96¸ì\xa0\x9cë\x82\x98 ë©\x80ê¸°ë§\x8c í\x96\x88ì\x97\x88ë\x8d\x98 ì\x8b\xa0ê¸°ë£¨ê°\x80 ë\x88\x88 ì\x95\x9eì\x97\x90 ì\x9e\x88ì\x96´ (ì\x97¬ê¸° ì\x9e\x88ì\x96´)\nA mirage that used to be ever so far awayÂ\xa0is in front of my eyes (Itâ\x80\x99s here) Iâ\x80\x99m a born singer ì\x96´ì©\x8cë©´ ì\x9d´ë¥¸ ê³\xa0ë°±\nIâ\x80\x99m a born singer,Â\xa0perhaps a bit early confession ê·¸ë\x9e\x98ë\x8f\x84 ë\x84\x88ë¬´ í\x96\x89ë³µí\x95´ Iâ\x80\x99m good\nBut, Iâ\x80\x99m so happy, Iâ\x80\x99m good Â\xa0 ë\x82\x9cì\x83\x9d ì²\x98ì\x9d\x8c ë°©í\x83\x84ì\x9d´ë\x9e\x80 ì\x9d´ë¦\x84ì\x9c¼ë¡\x9c ì\x84\xa0 ë¬´ë\x8c\x80\nThe stage that I stepped onto in the name of Bangtan for the first time in my life ì\x82¼ë\x85\x84 ì\xa0\x84 ì²« ë¬´ë\x8c\x80ì\x9d\x98 ë§\x88ì\

## Saving all the extracted lyrics with their song names in a JSON File

In [70]:
# Save the dictionary as a JSON file
with open('song_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(song_data, json_file, ensure_ascii=False, indent=2)